# Building your own algorithm container: Intel DAAL KMeans Example.

With Amazon SageMaker, you can package your own algorithms that can than be trained and deployed in the SageMaker environment. This notebook will guide you through an example that shows you how to build a Docker container for SageMaker and use it for training.

By packaging an algorithm in a container, you can bring almost any code to the Amazon SageMaker environment, regardless of programming language, environment, framework, or dependencies. 

1. [Building your own algorithm container](#Building-your-own-algorithm-container)
  1. [When should I build my own algorithm container?](#When-should-I-build-my-own-algorithm-container?)
  1. [Permissions](#Permissions)
  1. [The example](#The-example)
  1. [The presentation](#The-presentation)
1. [Part 1: Packaging and Uploading your Algorithm for use with Amazon SageMaker](#Part-1:-Packaging-and-Uploading-your-Algorithm-for-use-with-Amazon-SageMaker)
    1. [An overview of Docker](#An-overview-of-Docker)
    1. [How Amazon SageMaker runs your Docker container](#How-Amazon-SageMaker-runs-your-Docker-container)
      1. [Running your container during training](#Running-your-container-during-training)
        1. [The input](#The-input)
        1. [The output](#The-output)
      1. [Running your container during hosting](#Running-your-container-during-hosting)
    1. [The parts of the sample container](#The-parts-of-the-sample-container)
    1. [The Dockerfile](#The-Dockerfile)
    1. [Building and registering the container](#Building-and-registering-the-container)
  1. [Testing your algorithm on your local machine or on an Amazon SageMaker notebook instance](#Testing-your-algorithm-on-your-local-machine-or-on-an-Amazon-SageMaker-notebook-instance)
  1. [Testing the container locally with Iris Dataset](#Testing-the-container-locally-with-Iris-Dataset)
1. [Part 2: Training and Hosting your Algorithm in Amazon SageMaker](#Part-2:-Training-and-Hosting-your-Algorithm-in-Amazon-SageMaker)
  1. [Set up the environment](#Set-up-the-environment)
  1. [Create the session](#Create-the-session)
  1. [Upload the data for training](#Upload-the-data-for-training)
  1. [Create an estimator and fit the model](#Create-an-estimator-and-fit-the-model)
  1. [Download DAAL Trained Model and Compute Accuracy](#Download-DAAL-Trained-Model-and-Compute-Accuracy)
 

_or_ I'm impatient, just [let me see the code](#The-Dockerfile)!

## When should I build my own algorithm container?

You may not need to create a container to bring your own code to Amazon SageMaker. When you are using a framework (such as Apache MXNet or TensorFlow) that has direct support in SageMaker, you can simply supply the Python code that implements your algorithm using the SDK entry points for that framework. This set of frameworks is continually expanding, so we recommend that you check the current list if your algorithm is written in a common machine learning environment.

Even if there is direct SDK support for your environment or framework, you may find it more effective to build your own container. If the code that implements your algorithm is quite complex on its own or you need special additions to the framework, building your own container may be the right choice.

If there isn't direct SDK support for your environment, don't worry. You'll see in this walk-through that building your own container is quite straightforward.

## Permissions

Running this notebook requires permissions in addition to the normal `SageMakerFullAccess` permissions. This is because we'll creating new repositories in Amazon ECR. The easiest way to add these permissions is simply to add the managed policy `AmazonEC2ContainerRegistryFullAccess` to the role that you used to start your notebook instance. There's no need to restart your notebook instance when you do this, the new permissions will be available immediately.

## The example

Here, we'll show how to package a simple Python example which showcases the [K-Means][] algorithm from [Intel DAAL][] machine learning package. The example is purposefully fairly trivial since the point is to show the surrounding structure that you'll want to add to your own code so you can train and host it in Amazon SageMaker.

The ideas shown here will work in any language or environment. You'll need to choose the right tools for your environment to serve HTTP requests for inference, but good HTTP environments are available in every language these days.

In this example, we use a single image to support training and hosting (WIP). This is easy because it means that we only need to manage one image and we can set it up to do everything. Sometimes you'll want separate images for training and hosting because they have different requirements. Just separate the parts discussed below into separate Dockerfiles and build two images. Choosing whether to have a single image or two images is really a matter of which is more convenient for you to develop and manage.

If you're only using Amazon SageMaker for training or hosting, but not both, there is no need to build the unused functionality into your container.

[Intel DAAL]: https://software.intel.com/en-us/intel-daal
[K-Means]: https://software.intel.com/en-us/daal-programming-guide-k-means-clustering

## The presentation

This presentation is divided into two parts: _building_ the container and _using_ the container.

# Part 1: Packaging and Uploading your Algorithm for use with Amazon SageMaker

### An overview of Docker

If you're familiar with Docker already, you can skip ahead to the next section.

For many data scientists, Docker containers are a new concept, but they are not difficult, as you'll see here. 

Docker provides a simple way to package arbitrary code into an _image_ that is totally self-contained. Once you have an image, you can use Docker to run a _container_ based on that image. Running a container is just like running a program on the machine except that the container creates a fully self-contained environment for the program to run. Containers are isolated from each other and from the host environment, so the way you set up your program is the way it runs, no matter where you run it.

Docker is more powerful than environment managers like conda or virtualenv because (a) it is completely language independent and (b) it comprises your whole operating environment, including startup commands, environment variable, etc.

In some ways, a Docker container is like a virtual machine, but it is much lighter weight. For example, a program running in a container can start in less than a second and many containers can run on the same physical machine or virtual machine instance.

Docker uses a simple file called a `Dockerfile` to specify how the image is assembled. We'll see an example of that below. You can build your Docker images based on Docker images built by yourself or others, which can simplify things quite a bit.

Docker has become very popular in the programming and devops communities for its flexibility and well-defined specification of the code to be run. It is the underpinning of many services built in the past few years, such as [Amazon ECS].

Amazon SageMaker uses Docker to allow users to train and deploy arbitrary algorithms.

In Amazon SageMaker, Docker containers are invoked in a certain way for training and a slightly different way for hosting. The following sections outline how to build containers for the SageMaker environment.

Some helpful links:

* [Docker home page](http://www.docker.com)
* [Getting started with Docker](https://docs.docker.com/get-started/)
* [Dockerfile reference](https://docs.docker.com/engine/reference/builder/)
* [`docker run` reference](https://docs.docker.com/engine/reference/run/)

[Amazon ECS]: https://aws.amazon.com/ecs/

### How Amazon SageMaker runs your Docker container

Because you can run the same image in training or hosting, Amazon SageMaker runs your container with the argument `train` or `serve`. How your container processes this argument depends on the container:

* In the example here, we don't define an `ENTRYPOINT` in the Dockerfile so Docker will run the command `train` at training time and `serve` at serving time. In this example, we define these as executable Python scripts, but they could be any program that we want to start in that environment.
* If you specify a program as an `ENTRYPOINT` in the Dockerfile, that program will be run at startup and its first argument will be `train` or `serve`. The program can then look at that argument and decide what to do.
* If you are building separate containers for training and hosting (or building only for one or the other), you can define a program as an `ENTRYPOINT` in the Dockerfile and ignore (or verify) the first argument passed in. 

#### Running your container during training

When Amazon SageMaker runs training, your `train` script is run just like a regular Python program. A number of files are laid out for your use, under the `/opt/ml` directory:

    /opt/ml
    ├── input
    │   ├── config
    │   │   ├── hyperparameters.json
    │   │   └── resourceConfig.json
    │   └── data
    │       └── <channel_name>
    │           └── <input data>
    ├── model
    │   └── <model files>
    └── output
        └── failure

##### The input

* `/opt/ml/input/config` contains information to control how your program runs. `hyperparameters.json` is a JSON-formatted dictionary of hyperparameter names to values. These values will always be strings, so you may need to convert them. `resourceConfig.json` is a JSON-formatted file that describes the network layout used for distributed training. Since scikit-learn doesn't support distributed training, we'll ignore it here.
* `/opt/ml/input/data/<channel_name>/` (for File mode) contains the input data for that channel. The channels are created based on the call to CreateTrainingJob but it's generally important that channels match what the algorithm expects. The files for each channel will be copied from S3 to this directory, preserving the tree structure indicated by the S3 key structure. 
* `/opt/ml/input/data/<channel_name>_<epoch_number>` (for Pipe mode) is the pipe for a given epoch. Epochs start at zero and go up by one each time you read them. There is no limit to the number of epochs that you can run, but you must close each pipe before reading the next epoch.

##### The output

* `/opt/ml/model/` is the directory where you write the model that your algorithm generates. Your model can be in any format that you want. It can be a single file or a whole directory tree. SageMaker will package any files in this directory into a compressed tar archive file. This file will be available at the S3 location returned in the `DescribeTrainingJob` result.
* `/opt/ml/output` is a directory where the algorithm can write a file `failure` that describes why the job failed. The contents of this file will be returned in the `FailureReason` field of the `DescribeTrainingJob` result. For jobs that succeed, there is no reason to write this file as it will be ignored.

#### Running your container during hosting

Hosting has a very different model than training because hosting is reponding to inference requests that come in via HTTP. In this example, we use our recommended Python serving stack to provide robust and scalable serving of inference requests:

![Request serving stack](stack.png)

This stack is implemented in the sample code here and you can mostly just leave it alone. 

Amazon SageMaker uses two URLs in the container:

* `/ping` will receive `GET` requests from the infrastructure. Your program returns 200 if the container is up and accepting requests.
* `/invocations` is the endpoint that receives client inference `POST` requests. The format of the request and the response is up to the algorithm. If the client supplied `ContentType` and `Accept` headers, these will be passed in as well. 

The container will have the model files in the same place they were written during training:

    /opt/ml
    └── model
        └── <model files>



### The parts of the sample container

In the `container` directory are all the components you need to package the sample algorithm for Amazon SageMager:

    .
    ├── Dockerfile
    ├── build_and_push.sh
    └── kmeans
        ├── nginx.conf
        ├── predictor.py
        ├── serve
        ├── train
        └── wsgi.py

Let's discuss each of these in turn:

* __`Dockerfile`__ describes how to build your Docker container image. More details below.
* __`build_and_push.sh`__ is a script that uses the Dockerfile to build your container images and then pushes it to ECR. We'll invoke the commands directly later in this notebook, but you can just copy and run the script for your own algorithms.
* __`kmeans`__ is the directory which contains the files that will be installed in the container.
* __`local_test`__ is a directory that shows how to test your new container on any computer that can run Docker, including an Amazon SageMaker notebook instance. Using this method, you can quickly iterate using small datasets to eliminate any structural bugs before you use the container with Amazon SageMaker. We'll walk through local testing later in this notebook.

In this simple application, we only install five files in the container. You may only need that many or, if you have many supporting routines, you may wish to install more. These five show the standard structure of our Python containers, although you are free to choose a different toolset and therefore could have a different layout. If you're writing in a different programming language, you'll certainly have a different layout depending on the frameworks and tools you choose.

The files that we'll put in the container are:

* __`nginx.conf`__ is the configuration file for the nginx front-end. Generally, you should be able to take this file as-is.
* __`predictor.py`__ is the program that actually implements the Flask web server and the Kmeans predictions for this app (WIP). You'll want to customize the actual prediction parts to your application. Since this algorithm is simple, we do all the processing here in this file, but you may choose to have separate files for implementing your custom logic.
* __`serve`__ is the program started when the container is started for hosting. It simply launches the gunicorn server which runs multiple instances of the Flask app defined in `predictor.py`. You should be able to take this file as-is.
* __`train`__ is the program that is invoked when the container is run for training. You will modify this program to implement your training algorithm.
* __`wsgi.py`__ is a small wrapper used to invoke the Flask app. You should be able to take this file as-is.

In summary, the two files you will probably want to change for your application are `train` and `predictor.py`.

### The Dockerfile

The Dockerfile describes the image that we want to build. You can think of it as describing the complete operating system installation of the system that you want to run. A Docker container running is quite a bit lighter than a full operating system, however, because it takes advantage of Linux on the host machine for the basic operations. 

For the Python science stack, we will start from a standard Ubuntu installation and run the normal tools to install the things needed by scikit-learn. Finally, we add the code that implements our specific algorithm to the container and set up the right environment to run under.

Along the way, we clean up extra space. This makes the container smaller and faster to start.

Let's look at the Dockerfile for the example:

In [1]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Amazon AI <sage-learner@amazon.com>

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         bzip2 \
         python \
         nginx \
         tar \
         cpio \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN cd ~ && wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh && \
    chmod +x Miniconda3-latest-Linux-x86_64.sh && \
    bash Miniconda3-latest-Linux-x86_64.sh -b

RUN echo "export PATH=/root/miniconda3/bin:$PATH" > ~/.bashrc
RUN /bin/bash -c "source ~/.bashrc"
ENV PATH=/root/miniconda3/bin:$PATH

RUN conda create -y -n daal_py27 python=2.7

RUN conda install -y -n daal_py27 -c intel daal pydaal pandas
RUN conda install -y -n daal_py27 flask gevent gunicorn

RUN echo "source activate daal_py

### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. This code is also available as the shell script `container/build-and-push.sh`, which you can run as `build-and-push.sh daal-kmeans-sample` to build the image `daal-kmeans-sample`. 

This code looks for an ECR repository in the account you're using and the current default region (if you're using a SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it.

In [1]:
!cat container/build_and_push.sh

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
image=$1

if [ "$image" == "" ]
then
    echo "Usage: $0 <image-name>"
    exit 1
fi

chmod +x kmeans/train
chmod +x kmeans/serve

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${image}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
 

#### Building and pushing the container for the first time will take some time.

In [2]:
%%sh

cd container
./build_and_push.sh daal-kmeans-sample

Login Succeeded
Sending build context to Docker daemon  980.1MB
Step 1/18 : FROM ubuntu:16.04
16.04: Pulling from library/ubuntu
3b37166ec614: Pulling fs layer
504facff238f: Pulling fs layer
ebbcacd28e10: Pulling fs layer
c7fb3351ecad: Pulling fs layer
2e3debadcbf7: Pulling fs layer
2e3debadcbf7: Waiting
c7fb3351ecad: Waiting
ebbcacd28e10: Verifying Checksum
ebbcacd28e10: Download complete
504facff238f: Verifying Checksum
504facff238f: Download complete
3b37166ec614: Download complete
c7fb3351ecad: Verifying Checksum
c7fb3351ecad: Download complete
2e3debadcbf7: Download complete
3b37166ec614: Pull complete
504facff238f: Pull complete
ebbcacd28e10: Pull complete
c7fb3351ecad: Pull complete
2e3debadcbf7: Pull complete
Digest: sha256:45ddfa61744947b0b8f7f20b8de70cbcdd441a6a0532f791fd4c09f5e491a8eb
Status: Downloaded newer image for ubuntu:16.04
 ---> b9e15a5d1e1a
Step 2/18 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Running in 87a0fb26f853
Removing intermediate container 87a0f

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [3]:
!docker images

REPOSITORY                                                        TAG                 IMAGE ID            CREATED             SIZE
927702822156.dkr.ecr.us-west-2.amazonaws.com/daal-kmeans-sample   latest              b35d6a781540        4 minutes ago       2.9GB
daal-kmeans-sample                                                latest              b35d6a781540        4 minutes ago       2.9GB
ubuntu                                                            16.04               b9e15a5d1e1a        5 weeks ago         115MB


## Testing your algorithm on your local machine or on an Amazon SageMaker notebook instance

While you're first packaging an algorithm use with Amazon SageMaker, you probably want to test it yourself to make sure it's working right. In the directory `container/local_test`, there is a framework for doing this. It includes three shell scripts for running and using the container and a directory structure that mimics the one outlined above.

The scripts are:

* `train_local.sh`: Run this with the name of the image and it will run training on the local tree. You'll want to modify the directory `test_dir/input/data/...` to be set up with the correct channels and data for your algorithm. Also, you'll want to modify the file `input/config/hyperparameters.json` to have the hyperparameter settings that you want to test (as strings).
* `serve_local.sh`: Run this with the name of the image once you've trained the model and it should serve the model. It will run and wait for requests. Simply use the keyboard interrupt to stop it.
* `predict.sh`: Run this with the name of a payload file and (optionally) the HTTP content type you want. The content type will default to `text/csv`. For example, you can run `$ ./predict.sh payload.csv text/csv`.

The directories as shipped are set up to test the Kmeans sample algorithm presented here.

### Testing the container locally with Iris Dataset

In [4]:
%%time
from sklearn import datasets

# Load digits dataset
iris = datasets.load_iris()

# Create feature matrix
train_data = iris.data
print ("Training Data Shape: " + str(train_data.shape))

# Create target vector
train_labels = iris.target


Training Data Shape: (150, 4)
CPU times: user 368 ms, sys: 4.36 s, total: 4.72 s
Wall time: 358 ms


#### Save the training data in .csv format as required by your algorithm implementation

In [13]:
import numpy as np
training_data_file_prefix = 'container/local_test/test_dir/input/data/training/'
training_data_file = training_data_file_prefix + 'train_data_iris.csv'
np.savetxt(training_data_file, train_data, delimiter=",")

#### Use the train_local.sh to test training locally

In [12]:
%%sh
cd container/local_test
./train_local.sh daal-kmeans-sample

2018-10-15 15:51:16 INFO     Container setup completed, In Docker entrypoint - train... 
2018-10-15 15:51:16 INFO     Default Hyperparameters loaded: 
2018-10-15 15:51:16 INFO     {'accuracyThreshold': 0.0001,
 'assignFlag': True,
 'distanceType': 'euclidean',
 'gamma': 1.0,
 'initialCentroidMethod': 'defaultDense',
 'maxIterations': 300,
 'method': 'defaultDense',
 'nClusters': 2,
 'nRounds': 5,
 'oversamplingFactor': 0.5}
2018-10-15 15:51:16 INFO     Updated with user hyperparameters, Final Hyperparameters: 
2018-10-15 15:51:16 INFO     {'accuracyThreshold': 0.0001,
 'assignFlag': True,
 'distanceType': u'euclidean',
 'gamma': 1.0,
 'initialCentroidMethod': u'plusPlusDense',
 'maxIterations': 50,
 'method': u'defaultDense',
 'nClusters': 10,
 'nRounds': 5,
 'oversamplingFactor': 0.5}
2018-10-15 15:51:16 INFO     Reading training data... 
2018-10-15 15:51:16 INFO     Training Data Shape: (150, 4)
2018-10-15 15:51:16 INFO     Starting DAAL Kmeans training...
2018-10-15 15:51:16 INFO   

# Part 2: Training and Hosting your Algorithm in Amazon SageMaker

Once you have your container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [14]:

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [15]:
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sess = sage.Session()

## Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using some the classic [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [16]:
# S3 prefix
bucket_name = 'rpanchum'
data_key = 'kmeans_daal_iris'

output_location = 's3://{}/{}'.format(bucket_name, data_key)
print ("Training artifacts will be uploaded at: " + output_location)

data_location = sess.upload_data(training_data_file, bucket=bucket_name, key_prefix=data_key)

Training artifacts will be uploaded at: s3://rpanchum/kmeans_daal_iris


## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [17]:
daal_kmeans_image = '927702822156.dkr.ecr.us-west-2.amazonaws.com/daal-kmeans-sample:latest'

daal_kmeans = sage.estimator.Estimator(image_name=daal_kmeans_image,
                                       role=role,
                                       train_instance_count=1,
                                       train_instance_type='ml.c4.2xlarge',
                                       output_path=output_location,
                                       sagemaker_session=sess,
                                       hyperparameters={'nClusters': 3,
                                                        'initialCentroidMethod': 'plusPlusDense',
                                                        'maxIterations': 50 } 
                                      )

### Start DAAL Training

In [18]:
%%time
daal_kmeans.fit(data_location)

INFO:sagemaker:Creating training-job with name: daal-kmeans-sample-2018-10-15-15-58-08-911


2018-10-15 15:58:09 Starting - Starting the training job...
Launching requested ML instances......
Preparing the instances for training...
2018-10-15 15:59:50 Downloading - Downloading input data
2018-10-15 15:59:58 Training - Downloading the training image......
2018-10-15 16:00:59 Uploading - Uploading generated training model
2018-10-15 16:01:04 Completed - Training job completed

2018-10-15 16:00:54 INFO     Container setup completed, In Docker entrypoint - train... 
2018-10-15 16:00:54 INFO     Default Hyperparameters loaded: 
2018-10-15 16:00:54 INFO     {'accuracyThreshold': 0.0001,
 'assignFlag': True,
 'distanceType': 'euclidean',
 'gamma': 1.0,
 'initialCentroidMethod': 'defaultDense',
 'maxIterations': 300,
 'method': 'defaultDense',
 'nClusters': 2,
 'nRounds': 5,
 'oversamplingFactor': 0.5}
2018-10-15 16:00:54 INFO     Updated with user hyperparameters, Final Hyperparameters: 
2018-10-15 16:00:54 INFO     {'accuracyThreshold': 0.0001,
 'assignFlag': True,
 'distanceType': 

In [19]:
print("Training Job Name: ", daal_kmeans.latest_training_job.name)
print("Model saved at: ", daal_kmeans.model_data)

Training Job Name:  daal-kmeans-sample-2018-10-15-15-58-08-911
Model saved at:  s3://rpanchum/kmeans_daal_iris/daal-kmeans-sample-2018-10-15-15-58-08-911/output/model.tar.gz


### Download DAAL Trained Model and Compute Accuracy

In [20]:
import os, boto3

model_key = data_key + "/" + daal_kmeans.latest_training_job.name + '/output/model.tar.gz'
print ('Downloading the model saved at: ', daal_kmeans.model_data)
print(model_key)
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'daal_model.tar.gz')

os.system('tar -zxf daal_model.tar.gz')
daal_centroids, daal_assignments = np.load("daal-kmeans-model.npy", encoding = 'latin1')
print('DAAL centroids shape: ', daal_centroids.shape)

kmeans_daal_iris/daal-kmeans-sample-2018-10-15-15-58-08-911/output/model.tar.gz
DAAL centroids shape:  (3, 4)


In [22]:
from sklearn.cluster import KMeans 
from sklearn.metrics.cluster import v_measure_score

sklearn_kmeans = KMeans(3)
sklearn_kmeans.cluster_centers_ = daal_centroids

daal_train_assignments=sklearn_kmeans.predict(train_data)

print("DAAL Accuracy on Iris Train Set after 50 iterations: ", str(v_measure_score(train_labels, daal_train_assignments)))


DAAL Accuracy on Iris Train Set after 50 iterations:  0.7581756800057786
